In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from datetime import datetime

import geopandas as gpd
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import gee_data

In [4]:
model = keras.models.load_model('../models/44px_v2.9_2022-02-28.h5')

region = gpd.read_file('../data/boundaries/test_region.geojson').geometry[0].__geo_interface__

tile_size = 528
tile_padding = 16

chip_width = 44
chip_stride = 11

start_date = datetime(2023, 5, 1)
end_date = datetime(2023, 11, 1)
clear_threshold = 0.6

2023-11-30 17:32:16.459352: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2023-11-30 17:32:16.459377: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2023-11-30 17:32:16.459383: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2023-11-30 17:32:16.459579: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-30 17:32:16.459595: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:
# start a timer
start = datetime.now()
tiles = gee_data.create_tiles(region, tile_size, tile_padding)
print(f"{len(tiles)} tiles have been created")

data = gee_data.get_image_data(tiles, start_date, end_date, clear_threshold)
data = np.array([gee_data.pad_patch(patch, tile_size) for patch in data])
data_norm = np.array([gee_data.unit_norm(d) for d in data])

# convert the tiles to chips
chip_stack = []
chip_geom_stack = []
for tile_data, tile_geom in zip(data_norm, tiles):
    chips, chip_geoms = gee_data.chips_from_tile(tile_data, tile_geom, chip_width, chip_stride)
    chip_stack.extend(chips)
    chip_geom_stack.append(chip_geoms)

chip_stack = np.array(chip_stack)
chip_geom_stack = gpd.GeoDataFrame(pd.concat(chip_geom_stack, ignore_index=True), crs=tiles[0].crs)
print(f"Tiles have been broken into {len(chip_stack):,.0f} chips")

# make predictions and create a gdf
preds = model.predict(chip_stack)[:,1]
pred_df = gpd.GeoDataFrame(chip_geom_stack, crs=tiles[0].crs)
pred_df['preds'] = preds

# end the timer
end = datetime.now()

# print the time it took to run the pipeline
area_m2 = len(tiles) * (tile_size * 10) ** 2
# convert the meters squared to hectares
area_ha = area_m2 / 10000
duration = end - start
minutes, seconds = divmod(duration.total_seconds(), 60)
print(f"{area_ha:,.0f} hectares were analyzed in {minutes:.0f} minutes and {seconds:.0f} seconds")
print(f"At this speed, you could process an area the size of Rhode Island in {313900 * duration.total_seconds() / area_ha:.0f} seconds")
minutes, seconds = divmod(2203 * 313900 * duration.total_seconds() / area_ha, 60)
print(f"and the Amazon basin in {minutes / 60:,.1f} hours ({minutes / 60 / 24:,.1f} days)")

49 tiles have been created
Tiles have been broken into 99,225 chips


2023-11-30 17:34:04.591841: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


3101/3101 [==============================] - 7s 2ms/step
136,604 hectares were analyzed in 1 minutes and 19 seconds
At this speed, you could process an area the size of Rhode Island in 181 seconds
and the Amazon basin in 110.5 hours (4.6 days)


In [ ]:
# plot the chips in chip_stack where the preds are above a threshold
threshold = 0.9
positive_indices = np.where(preds > threshold)[0]
positive_chips = chip_stack[positive_indices]
print(len(positive_chips), 'chips with predictions above', threshold)
num_plots = int(np.ceil(np.sqrt(len(positive_chips))))
fig, axs = plt.subplots(num_plots, num_plots, figsize=(10,10))
for i, ax in enumerate(axs.flatten()):
    if i < len(positive_chips):
        ax.imshow(np.clip((positive_chips[i][:,:,(3,2,1)] + 2) / 5, 0, 1))
        ax.set_axis_off()
    else:
        ax.set_visible(False)
plt.show()